## Linkedin Scraper

- Referecen: https://github.com/tomquirk/linkedin-api
- SQLite Viewer: https://inloop.github.io/sqlite-viewer/


In [ ]:
!pip install linkedin-api

In [ ]:
!pip install python-dotenv

In [ ]:
from linkedin_api import Linkedin
import os
from dotenv import load_dotenv

load_dotenv()


# Authenticate using any Linkedin account credentials
api = Linkedin(os.getenv("EMAIL"), os.getenv("PWD"))

# GET a profile
profile = api.get_profile('chiphuyen')


# GET a profiles contact info
contact_info = api.get_profile_contact_info('chiphuyen')

In [ ]:
profile

In [ ]:
contact_info

In [ ]:
contact_info = api.get_profile_contact_info('chiphuyen')


In [ ]:
contact_info

## Get Posts

In [ ]:
posts = api.get_profile_posts('chiphuyen')


In [ ]:
len(posts)

In [ ]:
posts[0]

In [ ]:
url = posts[0]['socialContent']['shareUrl']
content =  c
like = posts[0]['socialDetail']['likes']['paging']['total']

In [ ]:
content

In [ ]:
url

In [ ]:
import pandas as pd

data = []
for post in posts:
    url = post['socialContent']['shareUrl']
    content = post['commentary']['text']['text']
    like = post['socialDetail']['likes']['paging']['total']
    data.append({'url': url, 'content': content, 'like': like})

df = pd.DataFrame(data)
df

In [ ]:
df.to_csv("huyenchip.csv",index=False)

## Save to database

## User Class

In [31]:
import sqlite3
import pandas as pd 

class User:
    def __init__(self, db='linkedin.db'):
        self.db = db
        
    def create_table(self):
        conn = sqlite3.connect(self.db)
        
        # Create a cursor object
        cursor = conn.cursor()

        # Create a table users
        cursor.execute('''CREATE TABLE IF NOT EXISTS users
                         (id INTEGER PRIMARY KEY, name TEXT)''')

        conn.close()
    def insert_user(self, name):
        conn = sqlite3.connect(self.db)
        cursor = conn.cursor()
        result = self.get_user_id(name)
        if result is None:
            cursor.execute("INSERT INTO users (name) VALUES (?)", (name,))
            
            conn.commit()
        else:
            print("user is existed")
        conn.close()

    def query(self):
        # Query the table
        conn = sqlite3.connect(self.db)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM users")
        rows = cursor.fetchall()
        
        for row in rows:
            print(row)

        conn.close()

        return rows

    def delete_user(self, user_id):
        """
        Deletes a user from the database by their ID.
        
        Args:
            user_id (int): The ID of the user to be deleted.
        """
        conn = sqlite3.connect(self.db)
        cursor = conn.cursor()
        
        # Delete the user from the table
        cursor.execute("DELETE FROM users WHERE id = ?", (user_id,))
        
        # Commit the changes and close the connection
        conn.commit()
        conn.close()

    def get_user_id(self,name):
        conn = sqlite3.connect(self.db)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM users where name = ?", (name,))
        result = cursor.fetchone()
        conn.close()

        if result is None:
            return None

        return result[0]

# user = User('linkedin.db')
# user.query()
# user.delete_user(2)
# user.query()
# user.insert_user('Hung Le')
# user.get_user_id('Huyen Chip')

### Create table posts

In [32]:
class Post:
    def __init__(self, db='linkedin.db'):
        self.db = db
        self.user = User(db)
        self.table_name = 'posts'
        
    def create_table(self):
        conn = sqlite3.connect(self.db)
        
        # Create a cursor object
        cursor = conn.cursor()

        cursor.execute('''CREATE TABLE IF NOT EXISTS posts
                 (id INTEGER PRIMARY KEY, user_id INTEGER, url TEXT, content TEXT, like INTEGER)''')
        
        conn.commit()

        conn.close()

    def get_urls(self):
        conn = sqlite3.connect(self.db)
        cursor = conn.cursor()
        cursor.execute("SELECT url FROM posts")
        urls = [row[0] for row in cursor.fetchall()]
        conn.close()
        return urls
        
    def insert_post(self, user_name: str, df: pd.DataFrame):
        conn = sqlite3.connect(self.db)
        user_id = self.user.get_user_id(user_name)
        if user_id is not None:
            df['user_id'] = user_id
            inserted_urls = self.get_urls()
            df = df[~df['url'].isin(inserted_urls)]
            df.to_sql(self.table_name, conn, if_exists='append', index=False)

    def query(self, is_show=False):
        # Query the table
        conn = sqlite3.connect(self.db)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM posts")
        rows = cursor.fetchall()

        if is_show:
            for row in rows:
                print(row)

        conn.close()

        return rows

    def to_df(self, items):
        """ Return the post into dataframe """
        list_data = []
        for item in items:
            list_data.append({"url": item[0], "content": item[1], "like": item[2], 'user_id': item[3]})
        return pd.DataFrame(list_data)
        

    def delete_post(self, post_id):
        """
        Deletes a user from the database by their ID.
        
        Args:
            user_id (int): The ID of the user to be deleted.
        """
        conn = sqlite3.connect(self.db)
        cursor = conn.cursor()
        
        # Delete the user from the table
        cursor.execute("DELETE FROM posts WHERE id = ?", (post_id,))
        
        conn.commit()
        conn.close()

    def delete_all(self):
        """
        Deletes all posts from the database.
        """
        conn = sqlite3.connect(self.db)
        cursor = conn.cursor()
        
        # Delete all posts from the table
        cursor.execute("DELETE FROM posts")
        
        # Commit the changes and close the connection
        conn.commit()
        conn.close()


# post_db = Post('linkedin.db')
# post_db.query()
# post_db.delete_all()


## Pipeline

Input: Linkin ID

Output: Save to database

In [37]:
from linkedin_api import Linkedin
import os
from dotenv import load_dotenv
import pandas as pd


load_dotenv()

INPUT_ID = 'chiphuyen'
# INPUT_ID = 'suvendu-pati-596757114'
POST_COUNT = 20

# Authenticate using any Linkedin account credentials
api = Linkedin(os.getenv("EMAIL"), os.getenv("PWD"))

posts = api.get_profile_posts(INPUT_ID, post_count=POST_COUNT)

data = []
for post in posts:
    url = post['socialContent']['shareUrl']
    content = post['commentary']['text']['text']
    like = post['socialDetail']['likes']['paging']['total']
    data.append({'url': url, 'content': content, 'like': like})

df = pd.DataFrame(data)

user_db = User('linkedin.db')
post_db = Post('linkedin.db')
user_db.create_table()
post_db.create_table()
user_db.insert_user(INPUT_ID)
post_db.insert_post(user_name=INPUT_ID, df=df)

user is existed


In [38]:
result_df = post_db.to_df(post_db.query())

In [39]:
result_df

,url,content,like,user_id
0,1,1,https://www.linkedin.com/posts/chiphuyen_mleng...,Building a platform for generative AI applicat...
1,2,1,https://www.linkedin.com/posts/chiphuyen_llms-...,"In many conversations, I noticed several commo..."
2,3,1,https://www.linkedin.com/posts/chiphuyen_snowf...,As an engineer who've learned so much from wri...
3,4,1,https://www.linkedin.com/posts/chiphuyen_gpu-d...,The rapid adoption of GPUs had made GPU optimi...
4,5,1,https://www.linkedin.com/posts/chiphuyen_aieng...,A big issue I see with AI systems is that peop...
5,6,1,https://www.linkedin.com/posts/chiphuyen_aieng...,LinkedIn has published one of the best reports...
6,7,1,https://www.linkedin.com/posts/chiphuyen_aieng...,I’m making a list of things to consider when u...
7,8,1,https://www.linkedin.com/posts/chiphuyen_aieng...,I have this hypothesis that the most popular e...
8,9,1,https://www.linkedin.com/posts/chiphuyen_aieng...,I’m excited to share that I’m working on a new...
9,10,1,https://www.linkedin.com/posts/chiphuyen_rag-w...,Absolutely loved the discussions and the energ...
